In [2]:
import numpy as np
import rasterio as rio
from pathlib import Path

from typing import List

In [25]:
def list_gfc_urls(
    x_min: int | float,
    x_max: int | float,
    y_min: int | float,
    y_max: int | float,
    product: str,
    step_size: int = 10,
    version: str ="GFC-2024-v1.12",
    root: str ="https://storage.googleapis.com/earthenginepartners-hansen",
    ) -> List[str]:

    # Check wether the chosen product is available
    available_products = ("treecover2000", "gain", "lossyear", "datamask", "first", "last")
    if product not in available_products:
        raise ValueError(f"'{product}' is not one of the supported products {available_products}")
    
    # Make the common header
    header = f"{root}/{version}/Hansen_{version}_{product}"

    # Make bouns to use during the iteration
    original_bounds = np.array([x_min, x_max, y_min, y_max])
    divided_bounds = original_bounds/step_size
    left, bottom = 10 * np.floor(divided_bounds[[0, 2]]).astype(int)
    right, top = 10 * np.ceil(divided_bounds[[1, 3]]).astype(int)

    # Iterate to create the tile names
    urls = []
    for row_idx in range(bottom, top, step_size):

        row_direction = "N" if row_idx >= 0 else "S"
        row_name = f"{abs(row_idx):02}{row_direction}"

        for col_idx in range(left, right, step_size):

            col_direction = "E" if col_idx >= 0 else "W"
            col_name = f"{abs(col_idx):03}{col_direction}"

            tile_url = f"{header}_{row_name}_{col_name}.tif"
            urls.append(tile_url)

    return urls

In [28]:
bounding_box = {
    "y_min": -19.6,
    "y_max": -17.1,
    "x_min": 32.9,
    "x_max": 34.4,
    }   

In [30]:
gfc_tiles = list_gfc_urls(**bounding_box, product="lossyear")